In [1]:
import pandas as pd
import numpy as np
from HFCs_emimTf2N_PR import configuration
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest
from pyomo.opt import TerminationCondition, SolverStatus

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [3]:
data = pd.read_csv('R32_emimTf2N_10_2.csv')
data2 = pd.read_csv('R125_emimTf2N_10_2.csv')

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Make outlet arrays
IL_x = np.zeros(len(data["1"]))
IL_y = np.zeros(len(data["1"]))
R32_x = np.zeros(len(data["1"]))
R32_y = np.zeros(len(data["1"]))
R125_x = np.zeros(len(data["1"]))
R125_y = np.zeros(len(data["1"]))

alpha1 = np.zeros(len(data["1"]))

m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.5) 
m1.fs.F101.inlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)

for i in range(len(data["0"])):
    
    m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m1.fs.properties.PR_kappa_A['R32', 'emimTf2N'].fix(data["1"].iloc[i])
    m1.fs.properties.PR_kappa_A['emimTf2N', 'R32'].fix(data["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['emimTf2N', 'R125'].fix(data2["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['R125', 'emimTf2N'].fix(data2["1"].iloc[i])

    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "emimTf2N"].fix(0.7) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.2) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.1)

    m1.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)

    try:
        m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m1, tee = False)

        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"])
            IL_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "emimTf2N"])
            R32_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            print(i)
            
            alpha1[i] = (R125_y[i]/R125_x[i])/(R32_y[i]/R32_x[i])

    except ValueError:
        alpha1[i] = 'NaN'
       
        
results = pd.DataFrame(alpha1)
results.to_csv('emimTf2N_alpha_10_2.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
print(alpha1)

[1.28544192 1.44855412 1.36578874 1.23573149 1.44557452 1.00526458
 1.45061049 1.44951872 1.42921018 1.37358962 1.43506884 1.42299
 1.40943511 1.41954006 1.43792195 1.4574031  1.43813384 1.4560003
 1.45536884 1.38202273 1.45503363 1.44832251 1.42338446 1.42140634
 1.42317602 1.44814969 1.42415129 1.43058863 1.35514539 1.41002568
 1.44401527 1.44467892 1.2727196  1.41795834 1.37512725 1.36812711
 1.36261991 1.37449609 1.41795461 1.415906   1.42291121 1.45976207
 1.31165903 1.3959729  1.43769682 1.44795987 1.41304118 1.40548169
 1.45279903 1.42050675 1.44619151 1.45238918 1.32390561 1.44956545
 1.45177331 1.44793481 1.41345803 1.45300837 1.43376384 1.41419967
 1.4326685  1.41267204 1.43124724 1.45476947 1.37315279 1.44440131
 1.43921076 1.42403466 1.41635469 1.4471042  1.45252317 1.45026031
 1.37815661 1.42240457 1.38437737 1.42207032 1.41869306 1.42232141
 1.41588493 1.34953726 1.30570514 1.3285346  1.43726073 1.3791001
 1.43777026 1.42234781 1.35830026 1.45673689 1.10635844 1.43913527
